In [329]:
import pandas as pd
import numpy as np

names=['Datum(UTC)','Nicht Erneuerbar (MW)', 'Erneuerbar (MW)', 'DE-LU Day Ahead EUR/MWh']
DayAheadPrices=pd.read_csv("Daten entsoe/energy-charts_Stromproduktion_und_Börsenstrompreise_in_Deutschland_2024.csv",skiprows=2,names=names)

In [330]:
#preis_menge=DayAheadPrices
DayAheadPrices['datum']=pd.to_datetime(DayAheadPrices['Datum(UTC)'])
DayAheadPrices=DayAheadPrices.set_index('datum')
preis_menge=DayAheadPrices

In [331]:
names=['Datum','Jahr', 'Monat', 'Tag','PV IK8']
IKPV=pd.read_csv("Daten IK8/2024 PV Daily.csv",skiprows=1,names=names,sep=';',decimal=',',parse_dates=['Datum'],date_format='%d.%m.%Y').fillna(0)
IKPV=IKPV.drop(columns=['Jahr', 'Monat', 'Tag'])
IKPV=IKPV.set_index('Datum')

In [332]:
IKStrom=pd.read_csv("Daten IK8/2024 Hist Strom.csv",date_format='ISO8601',parse_dates=True,decimal='.')
IKStrom['last_changed dt']=pd.to_datetime(IKStrom['last_changed']).dt.floor('h')
IKStrom=IKStrom.set_index('last_changed dt')
ent_filter=['sensor.haushalt_positive_active_energy_total','sensor.haushalt_negative_active_energy_total','sensor.waermepumpe_positive_active_energy_total','sensor.waermepumpe_negative_active_energy_total']
IKStrom=IKStrom[IKStrom['entity_id'].isin(ent_filter)]
IKStrom['state']=pd.to_numeric(IKStrom['state'])
IKStrom=IKStrom.groupby('entity_id').resample('h')['state'].agg(['min']).ffill()
IKStrom=IKStrom.reset_index().pivot(index=['last_changed dt'],columns='entity_id',values='min')
IKStrom=IKStrom.ffill()

In [333]:
preis_menge[['Haus H','Haus S','WP H','WP S']]=IKStrom.diff().shift(-1).fillna(0)
preis_menge['DT']=preis_menge.index.date
#preis_menge['WP PV']=preis_menge['Haus H']-preis_menge['WP H']


In [350]:
cnt = 0
gap_cnt=0
gap=[]
cols=['Haus S','Haus H']
colsw=['WP S','WP H']
names=['Uhrzeit','kW']
#cols=['Haus S','Haus H']
#gap_save=[]
# Fülle PV auf Haus
for index,row in preis_menge.iterrows():
    sv=""
    #sm = row['Haus S']+row['Haus H']
    sm = row[cols[0]]+row[cols[1]]
    smw = row[cols[0]]+row[cols[1]]
    if sm == 0:
        #print(index)
        cnt +=1
        gap_cnt += 1
        gap=gap+[index]
    else:
        if len(gap)>0:
            gap=gap+[index]
            gap=gap+[min(gap)-pd.DateOffset(hours=1)]
            #print(index,gap_cnt,cnt)
            day_before_gap=min(gap).floor('d')-pd.DateOffset(days=1)
            
            gap_df=pd.DataFrame(gap,columns=['Datum'])
            gap_days=pd.DataFrame(gap_df['Datum'].dt.date.unique(),columns=['Datum']).set_index('Datum')
            gap_days['PV IK8']=IKPV['PV IK8']
            
            # Wieviele % und wie viel PW muss insgesamt auf den Tag verteilt werden
            # anderer Ansatz:
            #
            # Summe PV und Haushaltsstrom wird in Vergleich gesetzt und Verbrauch HH wird über die Tage verteilt. 
            for gd_index,gd_row in gap_days.iterrows():
                #print()
                r=gd_index
                miss_IK_PV=pd.read_csv("Daten IK8/Wechselrichter_Tag_{}-{:02d}-{:02d}.csv".format(r.year,r.month,r.day),names=names,sep=';',decimal=',',skiprows=1)
                miss_IK_PV=miss_IK_PV.groupby(miss_IK_PV.index//4-1)['kW'].agg(['sum'])*1000/4  
                miss_IK_PV=miss_IK_PV[miss_IK_PV.index>=0]
                for miss_PV_index, miss_PV_row in miss_IK_PV.iterrows():
                    new_date=pd.to_datetime('{} {:02d}'.format(gd_index,miss_PV_index),format='%Y-%m-%d %H',utc=True)
                    gap_df.loc[gap_df['Datum']==new_date,['Wert']]=miss_PV_row['sum']
                    gap_df=gap_df.fillna(0)
            gap_sum_haus_s=sum(preis_menge[preis_menge.index.isin(gap_df['Datum'])]['Haus S'])
            gap_count_s=len(gap_df.index)
            gap_avg_haus_s=gap_sum_haus_s/gap_count_s
            gap_genug_pv=len(gap_df[gap_df['Wert']>gap_avg_haus_s])
            gap_fill_haus_s=round(gap_sum_haus_s/(gap_count_s-gap_genug_pv))
            gap_df['fill HaS']=(gap_fill_haus_s-gap_df['Wert']).clip(lower=0)
            gap_df['fill HaH']=(gap_df['Wert']-gap_fill_haus_s).clip(lower=0)

            for gapdf_index, gapdf_row in gap_df.iterrows():
                gd_datum=gapdf_row['Datum']
                preis_menge.loc[preis_menge.index==gd_datum,'Haus H']=gapdf_row['fill HaH']
                preis_menge.loc[preis_menge.index==gd_datum,'Haus S']=gapdf_row['fill HaS']
            
            save_gap=gap
            save_row=row
            gap=[]
            gap_cnt=0
            #print(row)
            #print(gap_days)

# Fülle Haus auf WP
cnt = 0
gap_cnt=0
gap=[]
for index,row in preis_menge.iterrows():
    sv=""
    smw = row[colsw[0]]+row[colsw[1]]    
    if smw == 0:
        #print(index)
        cnt +=1
        gap_cnt += 1
        gap=gap+[index]
    else:
        if len(gap)>0:
            gap=gap+[index]
            gap=gap+[min(gap)-pd.DateOffset(hours=1)]
            #print(index,gap_cnt,cnt)
            day_before_gap=min(gap).floor('d')-pd.DateOffset(days=1)
            
            gap_df=pd.DataFrame(gap,columns=['Datum'])
            gap_days=pd.DataFrame(gap_df['Datum'].dt.date.unique(),columns=['Datum']).set_index('Datum')
            gap_days['PV IK8']=IKPV['PV IK8']

            gap_sum_hh_h=sum(preis_menge[preis_menge.index.isin(gap_df['Datum'])]['Haus H'])
            gap_sum_hh_s=sum(preis_menge[preis_menge.index.isin(gap_df['Datum'])]['Haus S'])
            gap_sum_wp_h=sum(preis_menge[preis_menge.index.isin(gap_df['Datum'])]['WP H'])
            gap_sum_wp_s=sum(preis_menge[preis_menge.index.isin(gap_df['Datum'])]['WP S'])

            gap_verbrauch_wp=gap_sum_hh_h-gap_sum_hh_s+gap_sum_wp_s-gap_sum_wp_h
            gap_count_s=len(gap_df.index)
            gap_avg_wp_v=gap_verbrauch_wp/gap_count_s

            # Überlegen ...
            for gapdf_index, gapdf_row in gap_df.iterrows():
                #print(gapdf_index)
                gd_datum=gapdf_row['Datum']

                wert_Haus_H=sum(preis_menge[preis_menge.index==gd_datum]['Haus H'])
                wert_Haus_S=sum(preis_menge[preis_menge.index==gd_datum]['Haus S'])

                fill_WP_H=max(0,wert_Haus_H-gap_avg_wp_v-wert_Haus_S)
                fill_WP_S=max(0,gap_avg_wp_v-wert_Haus_H+wert_Haus_S)
                #if wert_Haus_H< gap_avg_wp_v:
                #    fill_WP_H=0
                #else:
                #    fill_WP_H
                
                preis_menge.loc[preis_menge.index==gd_datum,'WP H']=fill_WP_H
                preis_menge.loc[preis_menge.index==gd_datum,'WP S']=fill_WP_S

            gap=[]
            gap_cnt=0

            
            
preis_menge['WP']=preis_menge['WP S']-preis_menge['Haus S']+preis_menge['Haus H']-preis_menge['WP H']
preis_menge['Haus']=preis_menge['Haus S']
preis_menge['PV']=preis_menge['WP H']

preis_menge['EUR WP']   = preis_menge['WP']* preis_menge['DE-LU Day Ahead EUR/MWh']  /1000000
preis_menge['EUR Haus'] = preis_menge['Haus']* preis_menge['DE-LU Day Ahead EUR/MWh']/1000000
preis_menge['EUR PV']   = preis_menge['PV']* preis_menge['DE-LU Day Ahead EUR/MWh']  /1000000

print(cnt)

1


In [336]:
preis_menge[preis_menge.index.isin(gap_df['Datum'])]

,Datum(UTC),Nicht Erneuerbar (MW),Erneuerbar (MW),DE-LU Day Ahead EUR/MWh,Haus H,Haus S,WP H,WP S,DT
datum,,,,,,,,,
2024-12-29 10:00:00+00:00,2024-12-29T10:00+00:00,25311.50,28596.55,100.19,947.0,0.0,0.0,629.574468,2024-12-29
2024-12-29 11:00:00+00:00,2024-12-29T11:00+00:00,25349.24,29182.96,100.72,864.0,0.0,0.0,712.574468,2024-12-29
2024-12-29 12:00:00+00:00,2024-12-29T12:00+00:00,25204.39,28755.96,99.73,459.0,0.0,0.0,1117.574468,2024-12-29
2024-12-29 13:00:00+00:00,2024-12-29T13:00+00:00,24890.68,27167.70,98.44,177.0,0.0,0.0,1399.574468,2024-12-29
2024-12-29 14:00:00+00:00,2024-12-29T14:00+00:00,25386.78,25251.33,104.01,0.0,227.0,0.0,1803.574468,2024-12-29
2024-12-29 15:00:00+00:00,2024-12-29T15:00+00:00,26480.23,25175.90,121.73,0.0,436.0,0.0,2012.574468,2024-12-29
2024-12-29 16:00:00+00:00,2024-12-29T16:00+00:00,27152.05,25591.75,122.00,0.0,442.0,0.0,2018.574468,2024-12-29
2024-12-29 17:00:00+00:00,2024-12-29T17:00+00:00,28131.17,25555.26,126.11,0.0,442.0,0.0,2018.574468,2024-12-29
2024-12-29 18:00:00+00:00,2024-12-29T18:00+00:00,28803.74,26073.59,121.34,0.0,442.0,0.0,2018.574468,2024-12-29


In [341]:
preis_menge['Haus']=preis_menge['Haus H']

In [351]:
preis_menge

,Datum(UTC),Nicht Erneuerbar (MW),Erneuerbar (MW),DE-LU Day Ahead EUR/MWh,Haus H,Haus S,WP H,WP S,DT,WP,PV,Haus,EUR WP,EUR Haus,EUR PV
datum,,,,,,,,,,,,,,,
2023-12-31 23:00:00+00:00,2023-12-31T23:00+00:00,8378.54,41780.27,0.10,0.0,216.6,0.0,351.7,2023-12-31,135.1,0.0,216.6,0.000014,0.000022,0.0
2024-01-01 00:00:00+00:00,2024-01-01T00:00+00:00,8284.44,41524.81,0.01,0.0,329.0,0.0,1831.0,2024-01-01,1502.0,0.0,329.0,0.000015,0.000003,0.0
2024-01-01 01:00:00+00:00,2024-01-01T01:00+00:00,8282.03,41348.67,0.00,0.0,260.0,0.0,395.2,2024-01-01,135.2,0.0,260.0,0.000000,0.000000,0.0
2024-01-01 02:00:00+00:00,2024-01-01T02:00+00:00,8318.30,40365.41,-0.01,0.0,241.7,0.0,1730.2,2024-01-01,1488.5,0.0,241.7,-0.000015,-0.000002,-0.0
2024-01-01 03:00:00+00:00,2024-01-01T03:00+00:00,8309.99,40235.53,-0.03,0.0,304.4,0.0,1339.2,2024-01-01,1034.8,0.0,304.4,-0.000031,-0.000009,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 18:00:00+00:00,2024-12-31T18:00+00:00,21383.15,38916.54,67.77,0.0,1139.5,0.0,3385.2,2024-12-31,2245.7,0.0,1139.5,0.152191,0.077224,0.0
2024-12-31 19:00:00+00:00,2024-12-31T19:00+00:00,18557.03,40027.98,35.56,0.0,350.7,0.0,1189.2,2024-12-31,838.5,0.0,350.7,0.029817,0.012471,0.0
2024-12-31 20:00:00+00:00,2024-12-31T20:00+00:00,15779.81,40653.81,15.70,0.0,368.1,0.0,2621.2,2024-12-31,2253.1,0.0,368.1,0.035374,0.005779,0.0


In [321]:
sum(preis_menge[preis_menge.index==gd_datum]['Haus H'])

1237.300000000745

In [291]:
miss_IK_PV=pd.read_csv("Daten IK8/Wechselrichter_Tag_{}-{:02d}-{:02d}.csv".format(2024,3,19),names=names,sep=';',decimal=',',skiprows=1)

In [288]:
miss_IK_PV

,Uhrzeit,kW
0,00:15,0.0
1,00:30,0.0
2,00:45,0.0
3,01:00,0.0
4,01:15,0.0
...,...,...
91,23:00,0.0
92,23:15,0.0
93,23:30,0.0
94,23:45,0.0


In [352]:
preis_menge.to_csv('test.csv')